<font style='font-size:1.5em'>**Initial Data Analysis**</font>

<font style='font-size:1.2em'>Using Reddit CSV</font>

**Author: Sentipedia**  

**Course: DS105L Project** 

**DATE: 3/29/2023** 

---


## Imports 

### Importing Libraries 

In [1]:
# importing libraries for web scrapping and API acess
import requests
from bs4 import BeautifulSoup

# importing libraries for dataframes 
import pandas as pd
import numpy as np

# importing libraries for plots 
from plotnine import *

# importing libraries for ploting networks
import networkx as nx
import matplotlib.pyplot as plt

# Data

### Looking at the Data 

In [2]:
# import language model 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# downloading NLTK analyzers 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /Users/ap/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ap/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ap/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
df_reddit = pd.read_csv('reddit.csv', usecols=['comment', 'comment_id'])
df_reddit.head()

,comment,comment_id
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as right wing anti socialism.",1_2_1_4_1
1,"I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed be more hopeful and optimistic, but nah.",1_8_1_1
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in the last election. You can believe it what ever you want, but if you think the people who voted for Brexit and Tories are the ones detached from reality and not the leftists who can't make progressive policies that anyone wants to vote for then I don't know what to tell ya",1_11_1_1
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital C), but the way our elections work exaggerated results to imply that we are. The tories got 56% of the seats with 43.6% of the votes having gained 7.5% of seats with a 1.2 percentage point swing in votes. This is with a protest vote over brexit and the relentless vilification of Corbyn by the press. It wont be long before people hit back and vote against the tories but if Scotland do ever leave the union then it will be a lot less likely.",1_15
4,"Brexit, the word, is why it succeeded. Nobody would have enthusiastically supported ""the withdrawal of the UK from the European Union project"". In my opinion, the papers have a lot to be responsible/guilty for by running with the terminology.",2


# Using NLTK

In [4]:
df_reddit_nltk = df_reddit.head(10).copy()

analyzer = SentimentIntensityAnalyzer()
i = 0
df_reddit_nltk['pos'] = None
df_reddit_nltk['score'] = None

for comment in df_reddit_nltk['comment']:
     tokens = nltk.word_tokenize(df_reddit_nltk['comment'][i])
     df_reddit_nltk['pos'][i] = nltk.pos_tag(tokens)
     df_reddit_nltk['score'][i] = analyzer.polarity_scores(df_reddit_nltk['comment'][i])
     i = i + 1

df_reddit_nltk

,comment,comment_id,pos,score
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as right wing anti socialism.",1_2_1_4_1,"[(Agree, NNP), (., .), (With, IN), (the, DT), (Tories, NNPS), (pushing, VBG), (Brexit, NNP), (,, ,), (they, PRP), (have, VBP), (become, VBN), (a, DT), (resting, JJ), (place, NN), (for, IN), (racists, NNS), (as, RB), (well, RB), (as, IN), (right, JJ), (wing, VBG), (anti, JJ), (socialism, NN), (., .)]","{'neg': 0.22, 'neu': 0.606, 'pos': 0.174, 'compound': -0.296}"
1,"I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed be more hopeful and optimistic, but nah.",1_8_1_1,"[(I, PRP), (just, RB), (thought, VBD), (that, IN), (with, IN), (plenty, NN), (of, IN), (vaccinations, NNS), (coming, VBG), (in, IN), (and, CC), (a, DT), (Brexit, NNP), (deal, NN), (finally, RB), (secured, VBN), (wed, NN), (be, VB), (more, RBR), (hopeful, JJ), (and, CC), (optimistic, JJ), (,, ,), (but, CC), (nah, RB), (., .)]","{'neg': 0.063, 'neu': 0.705, 'pos': 0.232, 'compound': 0.5152}"
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in the last election. You can believe it what ever you want, but if you think the people who voted for Brexit and Tories are the ones detached from reality and not the leftists who can't make progressive policies that anyone wants to vote for then I don't know what to tell ya",1_11_1_1,"[(If, IN), (you, PRP), (do, VBP), (n't, RB), (listen, VB), (to, TO), (your, PRP$), (core, NN), (voter, NN), (base, NN), (,, ,), (they, PRP), (wo, MD), (n't, RB), (vote, VB), (for, IN), (you, PRP), (., .), (That, DT), ('s, VBZ), (what, WP), (labour, NN), (did, VBD), (in, IN), (the, DT), (last, JJ), (election, NN), (., .), (You, PRP), (can, MD), (believe, VB), (it, PRP), (what, WP), (ever, RB), (you, PRP), (want, VBP), (,, ,), (but, CC), (if, IN), (you, PRP), (think, VBP), (the, DT), (people, NNS), (who, WP), (voted, VBD), (for, IN), (Brexit, NNP), (and, CC), (Tories, NNPS), (are, VBP), (the, DT), (ones, NNS), (detached, VBN), (from, IN), (reality, NN), (and, CC), (not, RB), (the, DT), (leftists, NNS), (who, WP), (ca, MD), (n't, RB), (make, VB), (progressive, JJ), (policies, NNS), (that, IN), (anyone, NN), (wants, VBZ), (to, TO), (vote, VB), (for, IN), (then, RB), (I, PRP), (do, VBP), (n't, RB), (know, VB), (what, WP), (to, TO), (tell, VB), (ya, NN)]","{'neg': 0.025, 'neu': 0.959, 'pos': 0.016, 'compound': -0.1531}"
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital C), but the way our elections work exaggerated results to imply that we are. The tories got 56% of the seats with 43.6% of the votes having gained 7.5% of seats with a 1.2 percentage point swing in votes. This is with a protest vote over brexit and the relentless vilification of Corbyn by the press. It wont be long before people hit back and vote against the tories but if Scotland do ever leave the union then it will be a lot less likely.",1_15,"[(Much, JJ), (as, IN), (I, PRP), (agree, VBP), (with, IN), (your, PRP$), (potential, JJ), (projections, NNS), (,, ,), (we, PRP), (are, VBP), (not, RB), (overwhelmingly, RB), (Conservative, JJ), ((, (), (capital, JJ), (C, NNP), (), )), (,, ,), (but, CC), (the, DT), (way, NN), (our, PRP$), (elections, NNS), (work, VBP), (exaggerated, JJ), (results, NNS), (to, TO), (imply, VB), (that, IN), (we, PRP), (are, VBP), (., .), (The, DT), (tories, NNS), (got, VBD), (56, CD), (%, NN), (of, IN), (the, DT), (seats, NNS), (with, IN), (43.6, CD), (%, NN), (of, IN), (the, DT), (votes, NNS), (having, VBG), (gained, VBN), (7.5, CD), (%, NN), (of, IN), (seats, NNS), (with, IN), (a, DT), (1.2, CD), (percentage, NN), (point, NN), (swing, NN), (in, IN), (votes, NNS), (., .), (This, DT), (is, VBZ), (with, IN), (a, DT), (protest, NN), (vote, NN), (over, IN), (brexit, NN), (and, CC), (the, DT), (relentless, JJ), (vilification, NN), (of, IN), (Corbyn, NNP), (by, IN), (t

# Using DistilBERT Model

In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [9]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_0'

In [10]:
df_reddit_DistilBERT = df_reddit.head(10).copy()
i = 0
df_reddit_DistilBERT['predicted'] = None

for comment in df_reddit_DistilBERT['comment']:
     inputs = tokenizer(df_reddit_DistilBERT['comment'][i], return_tensors="pt")
     with torch.no_grad():
          logits = model(**inputs).logits

     predicted_class_id = logits.argmax().item()
     df_reddit_DistilBERT['predicted'][i] = model.config.id2label[predicted_class_id]
     i = i + 1

df_reddit_DistilBERT

,comment,comment_id,predicted
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as right wing anti socialism.",1_2_1_4_1,LABEL_1
1,"I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed be more hopeful and optimistic, but nah.",1_8_1_1,LABEL_1
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in the last election. You can believe it what ever you want, but if you think the people who voted for Brexit and Tories are the ones detached from reality and not the leftists who can't make progressive policies that anyone wants to vote for then I don't know what to tell ya",1_11_1_1,LABEL_1
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital C), but the way our elections work exaggerated results to imply that we are. The tories got 56% of the seats with 43.6% of the votes having gained 7.5% of seats with a 1.2 percentage point swing in votes. This is with a protest vote over brexit and the relentless vilification of Corbyn by the press. It wont be long before people hit back and vote against the tories but if Scotland do ever leave the union then it will be a lot less likely.",1_15,LABEL_1
4,"Brexit, the word, is why it succeeded. Nobody would have enthusiastically supported ""the withdrawal of the UK from the European Union project"". In my opinion, the papers have a lot to be responsible/guilty for by running with the terminology.",2,LABEL_1
5,"""Brexit means Brexit""... =""\n\nThe one stand out comment I remember from the leave campaign is Gove's ""the British people have had quite enough of experts."". I guess it was true :(",2_1_3,LABEL_1
6,"The word actually wasnt really used much anywhere until the week or so before the referendum, which is quite interesting for various reasons.\n\nhttps://trends.google.com/trends/explore?date=today%205-y&amp;geo=GB&amp;q=brexit",2_2,LABEL_1
7,"It was used, but only as a kind of novelty term - let's not forget that it was intended as a slightly mocking or irreverent riff off the original ""Grexit"" which referred to Greece leaving the euro at the height of their economic troubles - which in the end never came to pass. Once the referendum was over, and it became clear we would leave rather than just being a hypothetical outcome, ""Brexit"" came into more mainstream usage.",2_2_1_1_1_1,LABEL_1
8,"This is exact right - the portmanteau of Grexit led to the portmanteau of Brexit, and at the time was treated as an amusement, not as a precursor to what was to come.\n\nIf the leave vote contingency have said anything correct over the last 4+ years it is the accurate accusation the the Remain camp ran a truly terrible campaign that failed to recognise what the Leave campaign spotted - and that is that millions of non-voters could be encouraged to vote just once by stirring up enough emotional buy-in to a simple message.",2_2_1_1_1_1_1,LABEL_1
9,"Its probably somewhere between the two of us tbh.\n\nA Google search between 1st January 2016 and 31st May 2016 shows a few results in mainstream media but its only a page or so of bigger results until youre into mostly academic territory and nerdy politics/economics blogs. The Economist and some other sources were still using quote marks around the word in those months leading up to the referendum, which I think gives an idea as to its novelty at the time.\n\nhttps://www.google.co.uk/search?q=brexit&amp;hl=en-gb&amp;tbs=cdr:1,cd_min:1/1/2016,cd_max:5/31/2016",2_2_1_1_1_2,LABEL_1


In [12]:
text = "This is an example sentence to encode using DistilBERT."
encoded_text = tokenizer.encode("[CLS] " + text + " [SEP]")
input_ids = torch.tensor([encoded_text])
outputs = model(input_ids)
hidden_states = outputs[0]

In [13]:
input_ids

tensor([[  101,   101,  2023,  2003,  2019,  2742,  6251,  2000,  4372, 16044,
          2478,  4487, 16643, 23373,  1012,   102,   102]])

In [14]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1000, -0.0733]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Using DistilBERT Model (Cosine Similarity)

In [6]:
df_reddit_DistilBERT = df_reddit.head(100).copy()
df_reddit_DistilBERT

,comment,comment_id
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as right wing anti socialism.",1_2_1_4_1
1,"I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed be more hopeful and optimistic, but nah.",1_8_1_1
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in the last election. You can believe it what ever you want, but if you think the people who voted for Brexit and Tories are the ones detached from reality and not the leftists who can't make progressive policies that anyone wants to vote for then I don't know what to tell ya",1_11_1_1
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital C), but the way our elections work exaggerated results to imply that we are. The tories got 56% of the seats with 43.6% of the votes having gained 7.5% of seats with a 1.2 percentage point swing in votes. This is with a protest vote over brexit and the relentless vilification of Corbyn by the press. It wont be long before people hit back and vote against the tories but if Scotland do ever leave the union then it will be a lot less likely.",1_15
4,"Brexit, the word, is why it succeeded. Nobody would have enthusiastically supported ""the withdrawal of the UK from the European Union project"". In my opinion, the papers have a lot to be responsible/guilty for by running with the terminology.",2
...,...,...
95,"A hard brexit was defined (prior to the referendum) as us leaving the customs union, What would your definition be?",2_2_1_4_1_2_2
96,"Because the current deal (backstop) ties the UK to EU rules with the added setback of having left the EU and not having any say in EU rules. With that argument, it's clear to see why hard brexiters are against it - it is not a hard brexit at all.",2_2_1_4_1_2_2_1
97,"How is Mays deal a hard brexit? We stay in the customs union until 2020 until a trade deal is reach. If none is reached we stay in it, indefinitely. That's not a hard brexit.",2_2_1_4_1_4
98,"i get paid in dollars, and i am sick of explaining to brexiteers, no, this is not making me rich, it's just making what i have worth less and i'm paying more tax on it too",2_2_1_4_2_1_1_1


In [7]:
def get_reference(num):
    try:
        temp_array = []
        num_list = num.split("_")
    
        if len(num_list) > 1:
            num_list.pop()

        concat_num = '_'.join(num_list)
        data = {'reference_id': concat_num}
        df = pd.DataFrame(data, index=[0])    
        return df 
    except: 
        data = {'reference_id': 'NA'}
        df = pd.DataFrame(data, index=[0])    
        return df 

In [8]:
df_reference = pd.concat(get_reference(comment_id) for comment_id in df_reddit_DistilBERT['comment_id'])
df_reddit_df_reference

,reference_id
0,1_2_1_4
0,1_8_1
0,1_11_1
0,1
0,2
...,...
0,2_2_1_4_1_2
0,2_2_1_4_1_2_2
0,2_2_1_4_1
0,2_2_1_4_2_1_1


In [14]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

statement1 = "The sky is blue."
statement2 = "The sky is green."

encoded_statement1 = tokenizer.encode(statement1, return_tensors='pt')
encoded_statement2 = tokenizer.encode(statement2, return_tensors='pt')

with torch.no_grad():
    vector1 = model(encoded_statement1)[0][:, 0, :].numpy()
    vector2 = model(encoded_statement2)[0][:, 0, :].numpy()

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector1, vector2)
print(similarity)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[0.99577045]]
